In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
costs = []

with open("./data/day19.txt") as f:
    while line := f.readline():
        line = line.rstrip().split(': ')[1].split('. ')
        ore_robot = (int(line[0].split(' ')[-2]), 0, 0)
        clay_robot = (int(line[1].split(' ')[-2]), 0, 0)
        obsidian_robot = (int(line[2].split(' ')[-5]), int(line[2].split(' ')[-2]), 0)
        geode_robot = (int(line[3].split(' ')[-5]), 0, int(line[3].split(' ')[-2]))
        costs.append((ore_robot, clay_robot, obsidian_robot, geode_robot))
len(costs), costs[0]

(30, ((4, 0, 0), (4, 0, 0), (4, 14, 0), (3, 0, 16)))

In [3]:
@cache
def dp(ore_bots, clay_bots, obsidian_bots, geode_bots, ore, clay, obsidian, time_left, cost_ind):
    if time_left == 1:
        return geode_bots
    
    res = time_left*geode_bots
    
    bots = (ore_bots, clay_bots, obsidian_bots)
    resources = (ore, clay, obsidian)
    
    rounds = []
    for i in range(3):
        if resources[i] >= costs[cost_ind][0][i]:
            rounds.append(0)
            continue
        if bots[i] == 0:
            rounds.append(math.inf)
            continue
        rounds.append(math.ceil((costs[cost_ind][0][i]-resources[i])/bots[i]))
    rounds = max(rounds)
    if rounds < time_left-1 and ore_bots < max([costs[cost_ind][i][0] for i in range(4)]):
        res = max(res, (rounds+1)*geode_bots+dp(ore_bots+1, clay_bots, obsidian_bots, geode_bots,
                           ore+(rounds+1)*ore_bots-costs[cost_ind][0][0],
                           clay+(rounds+1)*clay_bots-costs[cost_ind][0][1],
                           obsidian+(rounds+1)*obsidian_bots-costs[cost_ind][0][2],
                           time_left-rounds-1, cost_ind))
    
    rounds = []
    for i in range(3):
        if resources[i] >= costs[cost_ind][1][i]:
            rounds.append(0)
            continue
        if bots[i] == 0:
            rounds.append(math.inf)
            continue
        rounds.append(math.ceil((costs[cost_ind][1][i]-resources[i])/bots[i]))
    rounds = max(rounds)
    if rounds < time_left-1 and clay_bots < max([costs[cost_ind][i][1] for i in range(4)]):
        res = max(res, (rounds+1)*geode_bots+dp(ore_bots, clay_bots+1, obsidian_bots, geode_bots,
                           ore+(rounds+1)*ore_bots-costs[cost_ind][1][0],
                           clay+(rounds+1)*clay_bots-costs[cost_ind][1][1],
                           obsidian+(rounds+1)*obsidian_bots-costs[cost_ind][1][2],
                           time_left-rounds-1, cost_ind))
    
    rounds = []
    for i in range(3):
        if resources[i] >= costs[cost_ind][2][i]:
            rounds.append(0)
            continue
        if bots[i] == 0:
            rounds.append(math.inf)
            continue
        rounds.append(math.ceil((costs[cost_ind][2][i]-resources[i])/bots[i]))
    rounds = max(rounds)
    if rounds < time_left-1 and obsidian_bots < max([costs[cost_ind][i][2] for i in range(4)]):
        res = max(res, (rounds+1)*geode_bots+dp(ore_bots, clay_bots, obsidian_bots+1, geode_bots,
                           ore+(rounds+1)*ore_bots-costs[cost_ind][2][0],
                           clay+(rounds+1)*clay_bots-costs[cost_ind][2][1],
                           obsidian+(rounds+1)*obsidian_bots-costs[cost_ind][2][2],
                           time_left-rounds-1, cost_ind))
    
    rounds = []
    for i in range(3):
        if resources[i] >= costs[cost_ind][3][i]:
            rounds.append(0)
            continue
        if bots[i] == 0:
            rounds.append(math.inf)
            continue
        rounds.append(math.ceil((costs[cost_ind][3][i]-resources[i])/bots[i]))
    rounds = max(rounds)
    if rounds < time_left-1:
        res = max(res, (rounds+1)*geode_bots+dp(ore_bots, clay_bots, obsidian_bots, geode_bots+1,
                           ore+(rounds+1)*ore_bots-costs[cost_ind][3][0],
                           clay+(rounds+1)*clay_bots-costs[cost_ind][3][1],
                           obsidian+(rounds+1)*obsidian_bots-costs[cost_ind][3][2],
                           time_left-rounds-1, cost_ind))
    
    return res

In [4]:
results = []
for i in range(len(costs)):
    dp.cache_clear()
    results.append(dp(1, 0, 0, 0, 0, 0, 0, 24, i))
print('DP results:', results)
print('Part 1 result:', sum([(i+1)*x for i, x in enumerate(results)]))

DP results: [0, 1, 0, 0, 1, 2, 2, 11, 0, 12, 0, 0, 0, 0, 0, 3, 0, 2, 3, 5, 2, 2, 4, 1, 9, 12, 3, 0, 4, 0]
Part 1 result: 1418


In [5]:
results2 = []
for i in range(3):
    dp.cache_clear()
    starttime = time.time()
    print('Starting', i+1)
    results2.append(dp(1, 0, 0, 0, 0, 0, 0, 32, i))
    print(f'Done with {i+1} in {time.time()-starttime:.2f}s.')
print()
print('DP results:', results2)
print('Part 2 result:', math.prod(results2))

Starting 1
Done with 1 in 3.06s.
Starting 2
Done with 2 in 8.07s.
Starting 3
Done with 3 in 6.58s.

DP results: [11, 22, 17]
Part 2 result: 4114
